In [ ]:
!pip install -U autogluon > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
textblob 0.19.0 requires nltk>=3.9, but you have nltk 3.8.1 which is incompatible.


In [ ]:
import pandas as pd
from autogluon.multimodal import MultiModalPredictor
from sklearn.model_selection import train_test_split
print("✅ Cài đặt và import thư viện hoàn tất.")

✅ Cài đặt và import thư viện hoàn tất.


In [ ]:
import pandas as pd
train_df = pd.read_csv('train.csv')

In [ ]:
train_df.head()

,ID,Title,Domain
0,1,"What is good in a decision tree, a large or a ...",Techniques
1,2,Training data only contains single positive label,Techniques
2,3,Calculating percentage contribution of a negat...,Techniques
3,4,Unable to open solution checker!,Hackathons
4,5,User Name Change,Misc


In [ ]:
train_data_for_split = train_df.drop(columns=['ID'])
train_data, val_data = train_test_split(
    train_data_for_split,
    test_size=0.2,
    random_state=42,
    stratify=train_data_for_split['Domain']
)

In [ ]:
print(f"Kích thước tập huấn luyện: {train_data.shape}")
print(f"Kích thước tập validation: {val_data.shape}")

Kích thước tập huấn luyện: (3076, 2)
Kích thước tập validation: (769, 2)


In [ ]:
from autogluon.tabular import TabularPredictor

# Khởi tạo TabularPredictor
predictor = TabularPredictor(
    label='Domain',
    eval_metric='f1_macro',
    path='ag_tabular_models'
)

# Huấn luyện với thời gian dài hơn
predictor.fit(
    train_data,
    time_limit=1800, # Tăng lên 1 giờ
    presets='best_quality'
)

# Đánh giá lại
print(predictor.evaluate(val_data))

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       10.44 GB / 12.67 GB (82.4%)
Disk Space Avail:   69.83 GB / 112.64 GB (62.0%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stack

{'f1_macro': 0.3303412994976663, 'accuracy': 0.6007802340702211, 'balanced_accuracy': np.float64(0.3177562493202544), 'mcc': np.float64(0.37797468462620626)}


In [ ]:
(predictor.leaderboard(val_data, extra_metrics = ['accuracy', 'f1_macro', 'log_loss', 'f1_weighted', 'roc_auc_ovr', 'roc_auc_ovr_weighted']))

,model,score_test,accuracy,f1_macro,log_loss,f1_weighted,roc_auc_ovr,roc_auc_ovr_weighted,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_r131_BAG_L1,0.360792,0.622887,0.360792,-1.165018,0.583861,0.752582,0.791677,0.368211,f1_macro,3.727224,1.772466,80.070307,3.727224,1.772466,80.070307,1,True,16
1,LightGBMLarge_BAG_L1,0.354550,0.616385,0.354550,-1.290135,0.578771,0.738981,0.781179,0.355598,f1_macro,1.945348,0.715106,73.606621,1.945348,0.715106,73.606621,1,True,13
2,LightGBM_BAG_L1,0.353625,0.629389,0.353625,-1.183306,0.591285,0.746883,0.790928,0.371621,f1_macro,1.390033,0.899752,50.267764,1.390033,0.899752,50.267764,1,True,5
3,LightGBMXT_BAG_L1,0.353289,0.621586,0.353289,-1.146596,0.582923,0.753888,0.796216,0.372930,f1_macro,1.633805,0.741805,49.274992,1.633805,0.741805,49.274992,1,True,4
4,ExtraTreesGini_BAG_L1,0.342834,0.603381,0.342834,-1.275617,0.562539,0.731196,0.775309,0.341930,f1_macro,0.218681,0.307950,2.220372,0.218681,0.307950,2.220372,1,True,9
5,RandomForestGini_BAG_L2,0.339991,0.626788,0.339991,-1.180552,0.581540,0.759449,0.787841,0.352797,f1_macro,6.802108,5.736055,718.184798,0.143604,0.343129,7.807041,2,True,22
6,LightGBMXT_BAG_L2,0.338163,0.618986,0.338163,-1.206500,0.575104,0.760463,0.794207,0.371288,f1_macro,8.081261,6.145927,862.079934,1.422758,0.753001,151.702177,2,True,20
7,ExtraTreesEntr_BAG_L1,0.335059,0.603381,0.335059,-1.296706,0.559191,0.719306,0.763867,0.336918,f1_macro,0.222063,0.288308,1.989075,0.222063,0.288308,1.989075,1,True,10
8,XGBoost_BAG_L1,0.331543,0.616385,0.331543,-1.167865,0.573588,0.749887,0.789636,0.374846,f1_macro,1.799506,0.897835,52.737899,1.799506,0.897835,52.737899,1,True,11
9,WeightedEnsemble_L2,0.331543,0.616385,0.331543,-1.167865,0.573588,0.749887,0.789636,0.374846,f1_macro,1.801736,0.901900,53.943334,0.002230,0.004065,1.205435,2,True,18


In [ ]:
from autogluon.core.utils import infer_column_types

column_types = infer_column_types(train_data)
print(column_types)

ImportError: cannot import name 'infer_column_types' from 'autogluon.core.utils' (/usr/local/lib/python3.11/dist-packages/autogluon/core/utils/__init__.py)

In [ ]:
from autogluon.multimodal import MultiModalPredictor

column_types = {
    'Title': 'text',
    'Domain': 'categorical',
}

predictor = MultiModalPredictor(
    label='Domain',
    problem_type='multiclass',
)

predictor.fit(
    train_data=train_data,
    tuning_data=val_data,
    column_types=column_types,
    time_limit=3600,
    presets='best_quality'
)


No path specified. Models will be saved in: "AutogluonModels/ag-20250627_093744"
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Pytorch Version:    2.6.0+cu124
CUDA Version:       12.4
Memory Avail:       7.25 GB / 12.67 GB (57.2%)
Disk Space Avail:   67.07 GB / 112.64 GB (59.5%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /content/AutogluonModels/ag-20250627_093744
    ```

INFO: Seed set to 0
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 12: 'val_accuracy' reached 0.17295 (best 0.17295), saving model to '/content/AutogluonModels/ag-20250627_093744/epoch=0-step=12.ckpt' as top 3


...

INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/content/AutogluonModels/ag-20250627_093744")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).


